We used https://fred.stlouisfed.org/ to download Canada's economic data that might play a role in Real Estate trends, price levels etc...
One can also use https://tradingeconomics.com/canada/interest-rate.

In the folder "Canadian Economy Data", you will find csv containing monthly data about unemployment, Central Bank real interest rates, CPI(inflation), M1 money supply, CAD-USD exchange rates.


In [1]:
import pandas as pd

In [2]:
hpi_df = pd.read_csv(r"C:\Users\hp\Desktop\Projects Coding\Affinity_Propagation_Canada_Real_Estate_Market\Processed Data\cities_hpi_df.csv")
hpi_df = hpi_df.drop(columns=["HPI_atlantic"], axis = 1)
hpi_df.head()


,DATE,LRUN64TTCAM156S
0,1981-01-01,7.4
1,1981-02-01,7.4
2,1981-03-01,7.3
3,1981-04-01,7.1
4,1981-05-01,7.2


In [13]:
#Unemployment dataframe
#We will drop all rows up to 1981-01-01

unemployment_df = pd.read_csv(r"C:\Users\hp\Desktop\Projects Coding\Affinity_Propagation_Canada_Real_Estate_Market\Canadian Economy Data\unemployment_LRUN64TTCAM156S.csv")
#
unemployment_df = unemployment_df.loc[(unemployment_df['DATE'] >= '1981-01-01' )]
#Renaming column for readable dataframe 
unemployment_df = unemployment_df.rename(columns={unemployment_df.columns[1]: 'unemployment_rate'})
#Resetting index without reassigning 
unemployment_df.reset_index(drop = True, inplace = True)
#Displaying dataframe with filtered date
unemployment_df.head()

,DATE,unemployment_rate
0,1981-01-01,7.4
1,1981-02-01,7.4
2,1981-03-01,7.3
3,1981-04-01,7.1
4,1981-05-01,7.2


In [14]:
#CPI Consumer Price Index All Items Dataframe
#We will drop all rows up to 1981-01-01
consumer_price_index_df = pd.read_csv(r"C:\Users\hp\Desktop\Projects Coding\Affinity_Propagation_Canada_Real_Estate_Market\Canadian Economy Data\consumer_price_index_all_items_Canada_CPALTT01CAM659N.csv")

consumer_price_index_df = consumer_price_index_df.loc[(consumer_price_index_df['DATE'] >= '1981-01-01' )]

#Renaming column for readable dataframe 
consumer_price_index_df = consumer_price_index_df.rename(columns={consumer_price_index_df.columns[1]: 'CPI_inflation'})
#Resetting index without reassigning 
consumer_price_index_df.reset_index(drop = True, inplace = True)
#Displaying dataframe with filtered date
consumer_price_index_df.head()

,DATE,CPI_inflation
0,1981-01-01,12.200957
1,1981-02-01,12.056738
2,1981-03-01,12.676056
3,1981-04-01,12.587413
4,1981-05-01,12.211982


In [15]:
#CAD-USD Rate dataframe
#We will drop all rows up to 1981-01-01
cadusd_rate_df = pd.read_csv(r"C:\Users\hp\Desktop\Projects Coding\Affinity_Propagation_Canada_Real_Estate_Market\Canadian Economy Data\cad_usd_Spot_Rate.csv")

cadusd_rate_df = cadusd_rate_df.loc[(cadusd_rate_df['DATE'] >= '1981-01-01' )]

#Renaming column for readable dataframe 
cadusd_rate_df = cadusd_rate_df.rename(columns={cadusd_rate_df.columns[1]: 'CAD-USD'})
#Resetting index without reassigning 
cadusd_rate_df.reset_index(drop = True, inplace = True)
#Displaying dataframe with filtered date
cadusd_rate_df.head()

,DATE,CAD-USD
0,1981-01-01,1.1909
1,1981-02-01,1.1984
2,1981-03-01,1.1914
3,1981-04-01,1.1910
4,1981-05-01,1.2010


In [16]:
#Interest Rate Central Bank dataframe
#We will drop all rows up to 1981-01-01
central_bank_interest_rate_df = pd.read_csv(r"C:\Users\hp\Desktop\Projects Coding\Affinity_Propagation_Canada_Real_Estate_Market\Canadian Economy Data\interest_rate_Central_Bank_CanadaIRSTCB01CAM156N.csv")

central_bank_interest_rate_df = central_bank_interest_rate_df.loc[(central_bank_interest_rate_df['DATE'] >= '1981-01-01' )]

#Renaming column for readable dataframe 
central_bank_interest_rate_df = central_bank_interest_rate_df.rename(columns={central_bank_interest_rate_df.columns[1]: 'Interest_Rate_Central_Bank'})
#Resetting index without reassigning 
central_bank_interest_rate_df.reset_index(drop = True, inplace = True)
#Displaying dataframe with filtered date
central_bank_interest_rate_df.tail()

,DATE,Interest_Rate_Central_Bank
495,2022-04-01,1.25
496,2022-05-01,1.25
497,2022-06-01,1.75
498,2022-07-01,2.75
499,2022-08-01,2.75


In [12]:
#Money SUpply M1 Dataframe
#We will drop all rows up to 1981-01-01
m1_df = pd.read_csv(r"C:\Users\hp\Desktop\Projects Coding\Affinity_Propagation_Canada_Real_Estate_Market\Canadian Economy Data\M1_Canada_MANMM101CAM189S.csv")
m1_df
# m1_df = -m1_df.loc[(m1_df['DATE'] >= '1981-01-01' )]
# #Resetting index without reassigning 
# m1_df.reset_index(drop = True, inplace = True)
# #Displaying dataframe with filtered date
# m1_df.tail()

,DATE,MANMM101CAM189S
0,1960-01-01,9.700000e+09
1,1960-02-01,9.700000e+09
2,1960-03-01,9.700000e+09
3,1960-04-01,9.800000e+09
4,1960-05-01,9.800000e+09
...,...,...
745,2022-02-01,1.643523e+12
746,2022-03-01,1.641170e+12
747,2022-04-01,1.623740e+12
748,2022-05-01,1.620052e+12
